# Slackbot-related work

In [ ]:
from trulens_evalchain.tru_db import Record, LocalTinyDB, TruDB, LocalSQLite
from trulens_evalchain import tru
from trulens_evalchain import tru_feedback
from IPython.display import JSON
from ipywidgets import widgets
import json

In [ ]:
# tru_feedback.huggingface_language_match(prompt="Hello there?", response="How are you?")

In [ ]:
# db = LocalTinyDB("slackbot.json")
#tru.init_db("slackbot.sql")
#db = LocalSQLite("slackbot.sql.db")
db = LocalSQLite()

In [ ]:
df, dff = db.get_records_and_feedback(chain_ids=[])

In [ ]:
df

In [ ]:
from pprint import PrettyPrinter
pp = PrettyPrinter(compact=True)

for i, row in df.iterrows():
    
    display(widgets.HTML(f"<b>Question:</b> {row.input}"))
    
    display(widgets.HTML(f"<b>Answer:</b> {row.output}"))
    
    details = json.loads(eval(row.details))

    display(widgets.HTML(str(details['chain']['combine_docs_chain']['llm_chain']['prompt']['template'])))
    
    for doc in details['chain']['combine_docs_chain']['_call']['args']['inputs']['input_documents']:
        display(widgets.HTML(f"""
        <div style="border: 1px solid black; padding: 5px;">
        <b>Context chunk</b>: {doc['page_content']}
        """))

        """<br/>

        <b>source</b>: {doc['metadata']['source']}
        </div>"""

    print()


In [ ]:
df = db.select(
    Record,
    Record.record_id,
    Record.chain_id,
    Record.chain._call.args.inputs.question,
    Record.chain._call.rets.answer)

In [ ]:
df

In [ ]:
for row_id, row in df.iterrows():
    record_id = row.record_id
    chain_id = row.chain_id

    main_question = row['Record.chain._call.args.inputs.question']
    main_answer = row['Record.chain._call.rets.answer']

    print(chain_id, record_id, main_question)
    """
    
    print(question, answer)

    # Run feedback function and get value

    feedback = tru.run_feedback_function(
        main_question, main_answer, [
            tru_feedback.get_not_hate_function(
                evaluation_choice='prompt',
                provider='openai',
                model_engine='moderation'
            ),
            tru_feedback.get_sentimentpositive_function(
                evaluation_choice='response',
                provider='openai',
                model_engine='gpt-3.5-turbo'
            ),
            tru_feedback.get_relevance_function(
                evaluation_choice='both',
                provider='openai',
                model_engine='gpt-3.5-turbo'
            )
        ]
    )
    print(f"will insert overall feedback for chain {chain_id}, record {record_id}")
    db.insert_feedback(record_id=record_id, chain_id=chain_id, feedback=feedback)
    """
    
    # display(JSON(row.Record))
    # print(row.Record['chain'])

    model_name = "gpt-3.5-turbo"

    """
    for page in TruDB.project(query=Record.chain.combine_docs_chain._call.args.inputs.input_documents, obj=row.Record):
        answer = page['page_content']
        feedback = tru.run_feedback_function(
            main_question,
            answer,
	        [
            tru_feedback.get_qs_relevance_function(
                evaluation_choice='prompt',
                provider='openai',
                model_engine=model_name
            )]
        )
        db.insert_feedback(record_id=record_id, chain_id=chain_id, feedback=feedback)

    """

    feedback = tru.run_feedback_function(
        main_question, main_answer, [
            tru_feedback.get_language_match_function(
                provider='huggingface'
            )
        ]
    )
    print(f"will insert language match feedback for chain {chain_id}, record {record_id}")
    db.insert_feedback(record_id=record_id, chain_id=chain_id, feedback=feedback)

    # feedback = tru.run_feedback_function(

    #for leaf in TruDB.leafs(row.Record):
    #    print(leaf)


In [ ]:
row.record_id

In [ ]:
feedback = {'openai_hate_function': 1.849137515819166e-05,
 'openai_sentimentpositive_feedback_function': 1,
 'openai_relevance_function': 10}

In [ ]:
db.insert_feedback(2, feedback)

In [ ]:
db.select(Record, table=db.feedbacks)

In [ ]:
model_name = "gpt-3.5-turbo"
feedback = tru.run_feedback_function(
    "Who is Piotr?",
    "Piotr Mardziel works on transparency and accountability in machine learning with applications to security, privacy, and fairness. He holds Bachelor’s and Master’s degrees from the Worcester Polytechnic Institute and a PhD in computer science from University of Maryland, College Park. He has conducted post-doctoral research at Carnegie Mellon University, as well as taught classes in trustworthy machine learning at Stanford University and machine learning privacy and security at Carnegie Mellon University.",
	 [
        tru_feedback.get_qs_relevance_function(
            evaluation_choice='prompt',
            provider='openai',
            model_engine=model_name
        )])

In [ ]:
feedback